In [1]:
using Pkg
Pkg.activate(".")

"/home/michel/git/structjump-demo/Project.toml"

In [2]:
include("acopf.jl")

acopf_initialPt_IPOPT (generic function with 1 method)

In [4]:
opfdata = opf_loaddata("data/case9")

OPFData(Bus[Bus(1, 3, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(2, 2, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(3, 2, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(4, 1, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(5, 1, 90.0, 30.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(6, 1, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(7, 1, 100.0, 35.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(8, 1, 0.0, 0.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9), Bus(9, 1, 125.0, 50.0, 0.0, 0.0, 1, 1.0, 0.0, 345.0, 1, 1.1, 0.9)], Line[Line(1, 4, 0.0, 0.0576, 0.0, 250.0, 250.0, 250.0, 0.0, 0.0, 1, -360.0, 360.0), Line(4, 5, 0.017, 0.092, 0.158, 250.0, 250.0, 250.0, 0.0, 0.0, 1, -360.0, 360.0), Line(5, 6, 0.039, 0.17, 0.358, 150.0, 150.0, 150.0, 0.0, 0.0, 1, -360.0, 360.0), Line(3, 6, 0.0, 0.0586, 0.0, 300.0, 300.0, 300.0, 0.0, 0.0, 1, -360.0, 360.0), Line(6, 7, 0.0119, 0.1008, 0.209, 150.0, 150.0, 150.0, 0.0, 0.0, 1, -360.0, 3